In [1]:
import sklearn
print(sklearn.__version__)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

1.6.1


## Read Train Dataset

In [2]:
preprocessed_df = pd.read_csv("..\Data\processed\preprocessed_train.csv")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\HP\AppData\Local\Temp\ipykernel_14072\492529711.py:1: SyntaxWarning: invalid escape sequence '\D'
  preprocessed_df = pd.read_csv("..\Data\processed\preprocessed_train.csv")


##  Train: Target variable  Split

In [3]:
x = preprocessed_df.drop(columns=["RainTomorrow"])
y = preprocessed_df["RainTomorrow"]

print(x.info(5))
print(y.info(5))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113754 entries, 0 to 113753
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       113754 non-null  float64
 1   MinTemp        113754 non-null  float64
 2   MaxTemp        113754 non-null  float64
 3   Rainfall       113754 non-null  float64
 4   WindGustDir    113754 non-null  float64
 5   WindGustSpeed  113754 non-null  float64
 6   WindDir9am     113754 non-null  float64
 7   WindDir3pm     113754 non-null  float64
 8   WindSpeed9am   113754 non-null  float64
 9   WindSpeed3pm   113754 non-null  float64
 10  Humidity9am    113754 non-null  float64
 11  Humidity3pm    113754 non-null  float64
 12  Pressure9am    113754 non-null  float64
 13  Pressure3pm    113754 non-null  float64
 14  Temp9am        113754 non-null  float64
 15  Temp3pm        113754 non-null  float64
 16  Year           113754 non-null  float64
 17  Month          113754 non-nul

## Train: Neural Network

In [4]:
print(y.value_counts())

RainTomorrow
0    88252
1    25502
Name: count, dtype: int64


In [5]:
pip install xgboost lightgbm catboost



   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/150.0 MB 2.4 MB/s eta 0:01:03
   ---------------------------------------- 1.3/150.0 MB 2.6 MB/s eta 0:00:58
   ---------------------------------------- 1.8/150.0 MB 2.5 MB/s eta 0:00:59
    --------------------------------------- 2.4/150.0 MB 2.6 MB/s eta 0:00:57
    --------------------------------------- 2.9/150.0 MB 2.5 MB/s eta 0:00:58
    --------------------------------------- 3.4/150.0 MB 2.5 MB/s eta 0:00:58
   - -------------------------------------- 4.2/150.0 MB 2.6 MB/s eta 0:00:57
   - -------------------------------------- 4.7/150.0 MB 2.6 MB/s eta 0:00:57
   - -------------------------------------- 5.0/150.0 MB 2.5 MB/s eta 0:00:59
   - -------------------------------------- 5.8/150.0 MB 2.6 MB/s eta 0:00:56
   - -------------------------------------- 6.3/150.0 MB 2.6 MB/s eta 0:00:56

In [10]:
import numpy as np
from sklearn.utils import class_weight

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

# Compute class weights
class_weights_array = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y),
    y=y
)
class_weights = dict(zip(np.unique(y), class_weights_array))
print("✅ Class weights:", class_weights)

# --------------------
# XGBoost
scale_pos_weight = class_weights[0] / class_weights[1]
xgb_model = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1
)
xgb_model.fit(x, y)
xgb_model.save_model(r"..\outputs\models\xgb_model.json")
print("✅ XGBoost model saved as xgb_model.json")

# --------------------
# LightGBM
lgb_model = lgb.LGBMClassifier(
    class_weight=class_weights,
    random_state=42,
    n_jobs=-1
)
lgb_model.fit(x, y)
lgb_model.booster_.save_model(r"..\outputs\models\lgb_model.txt")
print("✅ LightGBM model saved as lgb_model.txt")

# --------------------
# CatBoost
cat_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    verbose=0,
    random_seed=42,
    class_weights=[class_weights[0], class_weights[1]]
)
cat_model.fit(x, y)
cat_model.save_model(r"..\outputs\models\cat_model.cbm")
print("✅ CatBoost model saved as cat_model.cbm")

print("All models trained and saved!")


✅ Class weights: {0: 0.6444839777002221, 1: 2.2302956630852484}


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:05:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ XGBoost model saved as xgb_model.json
[LightGBM] [Info] Number of positive: 25502, number of negative: 88252
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007109 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2294
[LightGBM] [Info] Number of data points in the train set: 113754, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
✅ LightGBM model saved as lgb_model.txt
✅ CatBoost model saved as cat_model.cbm
All models trained and saved!


In [12]:
import pandas as pd
import numpy as np

# Load test data
df_test = pd.read_csv(r"..\Data\processed\preprocessed_test.csv")

# Save 'id' column for final output
ids = df_test['id']

# Prepare test features
X_final_test = df_test.drop(['id'], axis=1)

# --------------------
# Load XGBoost model and predict
import xgboost as xgb

xgb_model = xgb.XGBClassifier()
xgb_model.load_model(r"..\outputs\models\xgb_model.json")

xgb_preds = xgb_model.predict(X_final_test)
# If you want probabilities instead: xgb_model.predict_proba(X_final_test)[:,1]

# --------------------
# Load LightGBM model and predict
import lightgbm as lgb

lgb_model = lgb.Booster(model_file=r"..\outputs\models\lgb_model.txt")

# LightGBM Booster.predict returns probabilities, so convert to class
lgb_probs = lgb_model.predict(X_final_test)
lgb_preds = (lgb_probs > 0.5).astype(int)

# --------------------
# Load CatBoost model and predict
from catboost import CatBoostClassifier

cat_model = CatBoostClassifier()
cat_model.load_model(r"..\outputs\models\cat_model.cbm")

cat_preds = cat_model.predict(X_final_test).astype(int)

# --------------------
# Save predictions with id (example for XGBoost)
output_xgb = pd.DataFrame({'id': ids, 'RainTomorrow': xgb_preds})
output_xgb.to_csv(r"..\outputs\submissions\xgb_predictions.csv", index=False)

output_lgb = pd.DataFrame({'id': ids, 'RainTomorrow': lgb_preds})
output_lgb.to_csv(r"..\outputs\submissions\lgb_predictions.csv", index=False)

output_cat = pd.DataFrame({'id': ids, 'RainTomorrow': cat_preds})
output_cat.to_csv(r"..\outputs\submissions\cat_predictions.csv", index=False)

print("✅ Predictions saved for XGBoost, LightGBM, and CatBoost")


✅ Predictions saved for XGBoost, LightGBM, and CatBoost


## Test: Prepare Subission file using test Dataset

In [8]:
import pandas as pd
from tensorflow.keras.models import load_model

# Load the test data
df_test = pd.read_csv(r"..\Data\processed\preprocessed_test.csv")

# Save the 'id' column to merge later
ids = df_test['id']

# Drop unnecessary columns
X_final_test = df_test.drop(['id'], axis=1)

# Load the trained neural network model
X_final_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28439 entries, 0 to 28438
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Location       28439 non-null  float64
 1   MinTemp        28439 non-null  float64
 2   MaxTemp        28439 non-null  float64
 3   Rainfall       28439 non-null  float64
 4   WindGustDir    28439 non-null  float64
 5   WindGustSpeed  28439 non-null  float64
 6   WindDir9am     28439 non-null  float64
 7   WindDir3pm     28439 non-null  float64
 8   WindSpeed9am   28439 non-null  float64
 9   WindSpeed3pm   28439 non-null  float64
 10  Humidity9am    28439 non-null  float64
 11  Humidity3pm    28439 non-null  float64
 12  Pressure9am    28439 non-null  float64
 13  Pressure3pm    28439 non-null  float64
 14  Temp9am        28439 non-null  float64
 15  Temp3pm        28439 non-null  float64
 16  Year           28439 non-null  float64
 17  Month          28439 non-null  float64
 18  Day   

In [9]:
import pandas as pd
from tensorflow.keras.models import load_model

# Load the test data
df_test = pd.read_csv(r"..\Data\processed\preprocessed_test.csv")

# Save the 'id' column to merge later
ids = df_test['id']

# Drop unnecessary columns
X_final_test = df_test.drop(['id'], axis=1)

# Load the trained neural network model
model = load_model(r"..\outputs\models\nn_model.h5")

# Ensure input shape matches the training features (19 features expected)
assert X_final_test.shape[1] == 20, "Input data must have 19 features!"

# Predict probabilities
pred_probs = model.predict(X_final_test)

# Convert probabilities to class labels (0 or 1)
predictions = (pred_probs > 0.5).astype(int).flatten()

# Create submission DataFrame
submission = pd.DataFrame({
    'id': ids,
    'RainTomorrow': predictions
})

# Save to Excel

submission.to_csv(r"..\outputs\submissions\nn_model.csv", index=False)
print("✅ Neural Network predictions saved to submission_nn.csv")


889/889 ━━━━━━━━━━━━━━━━━━━━ 1s 579us/step
✅ Neural Network predictions saved to submission_nn.csv
